In [90]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [74]:
def metric_calculation(data):
    all_accuracies=[]
    all_hammings=[]
    all_proportions=[]
    l=int(len(data)//3)
    for i in tqdm(range(l)):
        accuracy=[]
        hamming=[]
        proportion=[]
        for j in range(i*3, i*3+3):
            gt_tokens=data[j][3].lower().split()
            decoded_tokens=data[j][4].lower().split()
            if len(gt_tokens)==0:
                continue                
            min_count = min(len(gt_tokens), len(decoded_tokens))
            gt_tokens=np.array(gt_tokens[:min_count]).reshape(-1,1)
            decoded_tokens=np.array(decoded_tokens[:min_count]).reshape(-1,1)
            hamming.append((gt_tokens != decoded_tokens).sum())
            accuracy.append(np.all(gt_tokens == decoded_tokens).item())
            proportion.append(np.isin(decoded_tokens, gt_tokens).mean())
        if len(accuracy) == 3:
            all_accuracies.append(accuracy)
        if len(hamming) == 3:
            all_hammings.append(hamming)
        if len(proportion) == 3:
            all_proportions.append(proportion)
    return all_accuracies, all_hammings, all_proportions

In [75]:
llama10=np.load('llama_10_epoch_memorization_attack_results.npy', allow_pickle=True)
llama1=np.load('llama_1_epoch_memorization_attack_results.npy', allow_pickle=True)

In [76]:
l10_acc,l10_hamming,l10_prop=metric_calculation(llama10)
l1_acc,l1_hamming,l1_prop=metric_calculation(llama1)

100%|███████████████████████████████████████| 531/531 [00:00<00:00, 3572.92it/s]


In [102]:
pd.DataFrame(
    np.stack([np.stack(l10_hamming).mean(axis=0), np.stack(l1_hamming).mean(axis=0)]),
    columns=['length=10','length=20','length=30'],
    index=['llama_10_epochs','llama_1_epochs']
)

,length=10,length=20,length=30
llama_10_epochs,29.077253,28.935622,25.914163
llama_1_epochs,31.875536,29.313305,26.236052


In [103]:
pd.DataFrame(
    np.stack([np.stack(l10_prop).mean(axis=0), np.stack(l1_prop).mean(axis=0)]),
    columns=['length=10','length=20','length=30'],
    index=['llama_10_epochs','llama_1_epochs']
)

,length=10,length=20,length=30
llama_10_epochs,0.422597,0.360278,0.263450
llama_1_epochs,0.255476,0.216534,0.175672
